In [1]:
# Install required libraries
!pip install dash plotly pandas
import pandas as pd
from dash import Dash, dcc, html, dash_table, Input, Output
import plotly.express as px
import plotly.graph_objects as go
from google.colab import files

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 29.0 MB/s eta 0:00:00


In [2]:
# Load Dataset
uploaded = files.upload()
file_name = next(iter(uploaded))
df = pd.read_csv(file_name)


Saving Heart Failure.csv to Heart Failure.csv


In [3]:
# Convert boolean columns to strings
bool_cols = ['anaemia','diabetes','high_blood_pressure','sex','smoking','DEATH_EVENT']
df[bool_cols] = df[bool_cols].astype(str)

In [4]:
# Initialize Dash App
app = Dash(__name__)

In [5]:
# Define Layout
app.layout = html.Div([
    html.H1("Heart Failure Clinical Analytics Dashboard",
           style={'textAlign': 'center', 'color': '#2c3e50', 'marginBottom': '30px'}),

    # Row 1: Mortality Overview
    html.Div([
        html.Div([
            html.H3("Total Patients"),
            html.P(f"{len(df):,}")
        ], className='card', style={'background': '#3498db'}),

        html.Div([
            html.H3("Mortality Rate"),
            html.P(f"{df['DEATH_EVENT'].value_counts()[1]/len(df)*100:.1f}%")
        ], className='card', style={'background': '#e74c3c'}),

        html.Div([
            html.H3("Avg Follow-up Days"),
            html.P(f"{df['time'].mean():.1f}")
        ], className='card', style={'background': '#2ecc71'}),

        html.Div([
            html.H3("High Risk Group"),
            html.P("Age 70+ & EF<35%")
        ], className='card', style={'background': '#f39c12'})
    ], className='card-container'),

    # Row 2: Patient Selector and Key Metrics
    html.Div([
        dcc.Dropdown(
            id='patient-selector',
            options=[{'label': f"Patient {i+1} (Age: {row['age']})", 'value': i}
                    for i, row in df.iterrows()],
            value=0,
            style={'width': '100%'}
        ),
        html.Div(id='patient-summary')
    ], style={'padding': '20px'}),

    # Row 3: Clinical Visualizations
    html.Div([
        dcc.Graph(id='survival-plot'),
        dcc.Graph(id='risk-factors')
    ], style={'columnCount': 2}),

    # Row 4: Biomarker Analysis
    html.Div([
        dcc.Dropdown(
            id='biomarker-selector',
            options=[
                {'label': 'Creatinine Phosphokinase', 'value': 'creatinine_phosphokinase'},
                {'label': 'Serum Creatinine', 'value': 'serum_creatinine'},
                {'label': 'Serum Sodium', 'value': 'serum_sodium'},
                {'label': 'Platelets', 'value': 'platelets'}
            ],
            value='serum_creatinine',
            style={'width': '50%'}
        ),
        dcc.Graph(id='biomarker-distribution')
    ]),

    # Data Table
    dash_table.DataTable(
        id='data-table',
        columns=[{'name': col, 'id': col} for col in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={
            'textAlign': 'left',
            'padding': '8px',
            'whiteSpace': 'normal'
        },
        filter_action='native'
    )
], style={'fontFamily': 'Arial', 'padding': '20px'})

/tmp/ipython-input-5-978346667.py:15: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [7]:
# Callbacks
@app.callback(
    Output('patient-summary', 'children'),
    Input('patient-selector', 'value')
)
def update_patient_summary(selected_idx):
    patient = df.iloc[selected_idx]
    return html.Div([
        html.H3(f"Patient {selected_idx+1} Profile"),
        html.Table([
            html.Tr([html.Td("Age:"), html.Td(f"{patient['age']} ({patient['age_group']})")]),
            html.Tr([html.Td("Ejection Fraction:"), html.Td(f"{patient['ejection_fraction']}%")]),
            html.Tr([html.Td("Survival Status:"),
                    html.Td(html.Span("Deceased", style={'color':'red'}) if patient['DEATH_EVENT'] == 'True'
                    else html.Span("Survived", style={'color':'green'}))]),
            html.Tr([html.Td("Key Risk Factors:"), html.Td(", ".join([
                "Anaemia" if patient['anaemia'] == 'True' else "",
                "Diabetes" if patient['diabetes'] == 'True' else "",
                "Hypertension" if patient['high_blood_pressure'] == 'True' else ""
            ]))])
        ], style={'width': '100%'})
    ])

In [8]:
@app.callback(
    Output('survival-plot', 'figure'),
    Input('patient-selector', 'value')
)
def update_survival_plot(_):
    fig = px.histogram(df, x='time', color='DEATH_EVENT',
                      nbins=20, barmode='overlay',
                      color_discrete_map={'True':'#e74c3c','False':'#2ecc71'},
                      title='Survival Time Distribution')
    fig.update_layout(showlegend=False)
    return fig

@app.callback(
    Output('risk-factors', 'figure'),
    Input('patient-selector', 'value')
)
def update_risk_factors(_):
    fig = px.box(df, x='DEATH_EVENT', y='ejection_fraction',
                color='DEATH_EVENT',
                color_discrete_map={'True':'#e74c3c','False':'#2ecc71'},
                title='Ejection Fraction by Mortality Status')
    fig.update_layout(showlegend=False)
    return fig

@app.callback(
    Output('biomarker-distribution', 'figure'),
    Input('biomarker-selector', 'value')
)
def update_biomarker_distribution(selected_biomarker):
    fig = px.scatter(df, x='age', y=selected_biomarker,
                    color='DEATH_EVENT',
                    color_discrete_map={'True':'#e74c3c','False':'#2ecc71'},
                    title=f'{selected_biomarker.replace("_"," ").title()} by Age',
                    hover_data=['ejection_fraction'])
    fig.update_traces(marker=dict(size=12, opacity=0.7))
    return fig


In [10]:
# Run App
app.run(mode='inline')

<IPython.core.display.Javascript object>